In [16]:
TUNING_DATA_PATHS = ["topic/topic_topic_fine_tuning_data__common_crawl_2025-02-23_08-18__filtered.csv",
                     "topic/topic_topic_fine_tuning_data__2025-02-21_16-50__filtered.csv"]  # "topic/topic_fine_tuning_data_extractive_2_15.csv"  # "topic/topic_fine_tuning_data_guided__02_11_processed.csv"
TUNING_DATA_PATHS_WITH_NONE = ["topic/fine_tuning_data__with_none__common_crawl_2025-02-23_08-18.csv",
                               "topic/common_corpus_noise_none_3_12.csv",
                               "topic/search_simplified.csv"
                               ]


In [17]:
!pwd


/Users/Rrando/Documents/GitHub/smart-tab-grouping/notebooks


In [18]:
!cd ..

In [19]:
TAB_GROUPING_BUCKET_NAME = "stage-fx-tab-grouping"


In [20]:
import sys
sys.path.append("/Users/Rrando/Documents/GitHub/smart-tab-grouping/src")


In [21]:
from util.secrets import load_env
from util.storage import download_bucket_to_file, download_bucket_to_csv
from util.shorten_topic_length import ShortenTopicLength
from dotenv import load_dotenv

In [22]:
load_dotenv()

True

In [23]:
load_env()

/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Loaded secrets from moz-fx-mozsoc-ml-nonprod
Loading env from .env


In [24]:
import pandas as pd

In [25]:
NOISE_TRAINING_DATA_SET_INDEX = 1
local_filename = "tuning_data.csv"


In [26]:
        training_files = TUNING_DATA_PATHS_WITH_NONE
        print(f"Using training files {training_files}")
        datasets = []
        for training_file in training_files:
            download_bucket_to_file(TAB_GROUPING_BUCKET_NAME, training_file, local_filename)
            datasets.append(pd.read_csv(local_filename, keep_default_na=False).fillna(""))
        datasets[NOISE_TRAINING_DATA_SET_INDEX] = datasets[NOISE_TRAINING_DATA_SET_INDEX].sample(n=500).reset_index(drop=True)  # reduce number a bit of None set

        topic_data = pd.concat(datasets, ignore_index=True).fillna("")
        topic_data = topic_data.drop_duplicates(subset=["input_titles"])

Using training files ['topic/fine_tuning_data__with_none__common_crawl_2025-02-23_08-18.csv', 'topic/common_corpus_noise_none_3_12.csv', 'topic/search_simplified.csv']


/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/

In [27]:
topic_data.value_counts("output")

output
None                      772
Adult Content              30
Jewelry                    29
Real Estate                27
Home Decor                 26
                         ... 
Financial Accounts          1
Financial Advice            1
Financial Basics            1
Financial Education         1
iPhone Troubleshooting      1
Name: count, Length: 4270, dtype: int64

In [28]:
topic_data["output"] = topic_data.apply(lambda a: a["before_none"] if a["before_none"] else a["output"], axis=1)

In [29]:
topic_data.value_counts("output")

output
None                      480
Adult Content              35
Jewelry                    29
Local News                 27
Real Estate                27
                         ... 
Finance App                 1
Finance Documentaries       1
Finance Resources           1
Finance Trends              1
iPhone Troubleshooting      1
Name: count, Length: 4448, dtype: int64

In [30]:
#shorten_boost = 0.08
#print(f"Shortening labels with setting {shorten_boost}")
#stl = ShortenTopicLength(shorten_boost)
#topic_data = stl.shorten_topics(topic_data)


In [31]:
topic_data.value_counts("output")

output
None                      480
Adult Content              35
Jewelry                    29
Local News                 27
Real Estate                27
                         ... 
Finance App                 1
Finance Documentaries       1
Finance Resources           1
Finance Trends              1
iPhone Troubleshooting      1
Name: count, Length: 4448, dtype: int64

In [32]:
topic_data.value_counts("output").to_csv("total.csv")

In [33]:
topic_data.value_counts("output")

output
None                      480
Adult Content              35
Jewelry                    29
Local News                 27
Real Estate                27
                         ... 
Finance App                 1
Finance Documentaries       1
Finance Resources           1
Finance Trends              1
iPhone Troubleshooting      1
Name: count, Length: 4448, dtype: int64

In [71]:
topic_data

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input_titles,input_keywords,input_description,output,output_orig,before_none,url,shortened,input_with_prompt
0,0.0,0.0,0,Episode 16 - The plane show (Preview) - SomeNe...,,Steve N Allen and Erich McElroy this week get ...,Podcast Preview,Podcast Preview,Podcast Preview,,,Topic from keywords: . titles: \nEpisode 16 - ...
1,1.0,1.0,1,55wordstory: Theme 76 - Map/Maps,,"From @DayaDarwazaTodo: Questions everywhere, ...",Creative Writing,Creative Writing,Creative Writing,,,Topic from keywords: . titles: \n55wordstory: ...
2,2.0,2.0,2,a Fair Substitute for Heaven: Where Rachel McM...,,From Walter's letter to Rilla as he fights fo...,Personal Blog,Personal Blog,Personal Blog,,,Topic from keywords: . titles: \na Fair Substi...
3,3.0,3.0,3,Gravitational Waves | Biotechnology & MedTech,,,Science,Science & Technology,Science,,,Topic from keywords: . titles: \nGravitational...
4,4.0,4.0,4,Omega Speedmaster Moonwatch Co-Axial Moonphase...,,Omega Speedmaster Moonwatch Co-Axial Moonphase...,Luxury Watches,Luxury Watches,Luxury Watches,,,Topic from keywords: . titles: \nOmega Speedma...
...,...,...,...,...,...,...,...,...,...,...,...,...
6638,,95.0,95,instagram privacy settings at DuckDuckGo,,,Privacy Settings,,,,Settings,Topic from keywords: . titles: \ninstagram pri...
6639,,96.0,96,furniture stores near me at DuckDuckGo,,,Local Furniture Stores,,,,Furniture Stores,Topic from keywords: . titles: \nfurniture sto...
6640,,97.0,97,tutorials for photoshop beginners at DuckDuckGo,,,Photoshop Tutorials,,,,Photoshop,Topic from keywords: . titles: \ntutorials for...
6641,,98.0,98,tourist attractions in paris - Google Search,,,Paris Tourism,,,,Paris,Topic from keywords: . titles: \ntourist attra...


In [34]:
import sys
sys.path.append("/Users/Rrando/Documents/GitHub/smart-tab-grouping/src/jobs")



In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from distill_t5 import DistillTopicT5
from TuneGenTopicModel import create_trainer_for_config

In [37]:
configs = [
            {
                "learning_rate": 5e-3,
                "batch_size": 8,
                "model_name": "google/t5-efficient-tiny",
                "label_column": "output",
                "use_keywords": True,
                "single_tab_handling": False,
                "learning_rate_decay": False,
                "shorten_training_label_boost": 0.08,
                "teacher_model_artifact": "moso/tab_grouping/model-uxfe87sy:v0"
            }]

In [38]:
trainer = create_trainer_for_config(configs[0])

In [39]:
SINGLE_TAB_VALIDATION_PATH = "topic/single_tab_validation.csv"


In [40]:
        validation_data = download_bucket_to_csv(TAB_GROUPING_BUCKET_NAME, SINGLE_TAB_VALIDATION_PATH)


/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/Rrando/Documents/GitHub/smart-tab-grouping/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [41]:
trainer.setup_data(topic_data, validation_data)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Training Dataset size 5298
Eval Dataset size 1325
Validation Dataset size 36


In [42]:
trainer.device = "cpu"

model.safetensors:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

In [44]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    TrainingArguments, Trainer
)


In [45]:
teacher_model = T5ForConditionalGeneration.from_pretrained("google/t5-efficient-tiny")


In [68]:
tokenizer = T5Tokenizer.from_pretrained("google/t5-efficient-tiny")

In [50]:
import torch


In [51]:
        tokenized_training_dataset = trainer.train_dataset.map(trainer.preprocess_function, batched=True)
        tokenized_eval_dataset = trainer.eval_dataset.map(trainer.preprocess_function, batched=True)


Map:   0%|          | 0/5298 [00:00<?, ? examples/s]

Map:   0%|          | 0/1325 [00:00<?, ? examples/s]

In [52]:

        def add_decoder_ids(item_input_dict):
            decoder_input_ids = teacher_model._shift_right(torch.tensor(item_input_dict["labels"]))
            item_input_dict["decoder_input_ids"] = decoder_input_ids
            return item_input_dict

        tokenized_training_dataset = tokenized_training_dataset.map(add_decoder_ids, batched=True)
        tokenized_training_dataset.set_format(
            type="torch", columns=["input_ids", "attention_mask", "labels", "decoder_input_ids"]
        )

Map:   0%|          | 0/5298 [00:00<?, ? examples/s]

In [57]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
            tokenized_training_dataset,
            batch_size=4,
            shuffle=True,
        )


In [65]:
for a in train_loader:
    res = teacher_model(**a)
    text = teacher_model.generate(a["input_ids"])
    break

In [130]:
def preprocess_function(examples, teacher_model=None):
    # Tokenize the input text
    inputs = examples["input_text"]
    model_inputs = tokenizer(
        inputs, 
        max_length=512, 
        truncation=True, 
        padding="max_length"
    )

    if teacher_model is None:
        targets = examples["target_text"]
        with tokenizer.as_target_tokenizer():
            tokenized_targets = tokenizer(
                targets, 
                max_length=64, 
                truncation=True, 
                padding="max_length"
            )
        model_inputs["labels"] = tokenized_targets["input_ids"]
    else:
        # Generate predictions from teacher and re-tokenize to get padding etc
        generated_ids = teacher_model.generate(
            tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).input_ids,
            max_length=64
        )
        decoded_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        with tokenizer.as_target_tokenizer():
            tokenized_labels = tokenizer(
                decoded_outputs, 
                max_length=64, 
                truncation=True, 
                padding="max_length"
            )
        model_inputs["labels"] = tokenized_labels["input_ids"]
    model_inputs["labels"] = [
            [(token if token != tokenizer.pad_token_id else -100) for token in label_seq]
            for label_seq in model_inputs["labels"]
        ]

    return model_inputs

In [131]:
from functools import partial


In [132]:
tokenized_training_dataset = trainer.train_dataset.map(partial(preprocess_function, teacher_model=teacher_model), batched=True)


Map:   0%|          | 0/5298 [00:00<?, ? examples/s]

In [134]:
tokenized_training_dataset[:1]

{'input_text': ['Topic from keywords: . titles: \nLC/PC duplex with shutter Fiber Optical Adapter - Buy fiber optic coupler, optical fiber adapter, fiber adapter Product on Kewei fiber'],
 'target_text': ['Fiber Optic Products'],
 'input_ids': [[18059,
   45,
   12545,
   10,
   3,
   5,
   8342,
   10,
   3,
   6480,
   87,
   4051,
   146,
   9247,
   28,
   18094,
   22505,
   3,
   9546,
   1489,
   3,
   14808,
   49,
   3,
   18,
   4783,
   6433,
   18310,
   1158,
   52,
   6,
   14171,
   6433,
   3374,
   49,
   6,
   6433,
   3374,
   49,
   6246,
   30,
   2566,
   1123,
   23,
   6433,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [55]:
teacher_model(tokenized_training_dataset[:2])

AttributeError: 'dict' object has no attribute 'size'

ValueError: Columns ['decoder_input_ids'] not in the dataset. Current columns in the dataset: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels']

In [28]:
trainer.train()

wandb: Currently logged in as: moso-wandb-bot (moso). Use `wandb login --relogin` to force relogin


W&B Run ID: yecrzkc9
W&B Run Name: astral-salad-13


wandb: Downloading large artifact model-uxfe87sy:v0, 944.47MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:3.8


Map:   0%|          | 0/5306 [00:00<?, ? examples/s]

/Users/Rrando/Library/Caches/pypoetry/virtualenvs/mozsocial-ml-services-Hn4MZpip-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1327 [00:00<?, ? examples/s]

Map:   0%|          | 0/5306 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Epoch 0:   2%|██▏                                                                                      | 16/664 [00:49<33:44,  3.12s/it, loss_value=163]


KeyboardInterrupt: 

In [ ]:
tokenized_eval_dataset = trainer.eval_dataset.map(trainer.preprocess_function, batched=True)

In [ ]:
item = tokenized_eval_dataset[0]

In [ ]:
            input = item["input_text"]
            label = item['target_text']


In [ ]:
input

In [ ]:
label

In [ ]:
            input_encodings = trainer.tokenizer(input, return_tensors="pt", truncation=True,
                                             padding=True)
            label_encodings = trainer.tokenizer(label, return_tensors="pt", truncation=True,
                                             padding=True)


In [ ]:
input_encodings

In [ ]:
label_encodings

In [ ]:
                outputs = trainer.model(
                    input_ids=input_encodings["input_ids"],
                    attention_mask=input_encodings["attention_mask"],
                    labels=label_encodings["input_ids"]
                )


In [ ]:
outputs

In [ ]:
outputs_gen = trainer.model.generate(input_encodings["input_ids"], max_length=30, num_return_sequences=1)

In [ ]:
outputs_gen

In [ ]:
len(trainer.tokenizer)

In [ ]:
trainer.model.config.vocab_size

In [ ]:
trainer.tokenizer.vocab_size

In [ ]:
trainer.run_eval(tokenized_eval_dataset)

In [ ]:
   def run_eval(eval_dataset, name="Eval", prefix=None, log_wandb=True):
        results_labels = []
        results_output = []
        losses = []
        for item in eval_dataset:
            input = item["input_text"]
            label = item['target_text']
            input_encodings = trainer.tokenizer(input, return_tensors="pt", truncation=True,
                                             padding=True)
            label_encodings = trainer.tokenizer(label, return_tensors="pt", truncation=True,
                                             padding=True)
            input_ids = input_encodings["input_ids"]
            attention_mask = input_encodings["attention_mask"]
            label_ids = label_encodings["input_ids"]
            outputs = trainer.model.generate(input_ids, max_length=30, num_return_sequences=1)
            print(outputs)
            response = trainer.tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
trainer.model.resize_token_embeddings(len(trainer.tokenizer))


In [ ]:
run_eval(tokenized_eval_dataset)

In [ ]:
        tokenized_eval_dataset = trainer.eval_dataset.map(trainer.preprocess_function, batched=True)


In [ ]:
import wandb

In [ ]:
run = wandb.init(project="tab_grouping_distillation")

In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    TrainingArguments, Trainer
)


In [ ]:
artifact = run.use_artifact("moso/tab_grouping/model-uxfe87sy:v0", type='model')
artifact_dir = artifact.download()
teacher_model = T5ForConditionalGeneration.from_pretrained(artifact_dir)


In [ ]:
teacher_tokenizer = T5Tokenizer.from_pretrained("/Users/Rrando/Documents/GitHub/smart-tab-grouping/models/noble-yogurt-330")

In [ ]:
teacher_tokenizer.encode("TEst")

In [ ]:
len(teacher_tokenizer)

In [ ]:
trainer.tokenizer.encode("TEst")

In [ ]:
teacher_tokenizer = T5Tokenizer.from_pretrained("moso/tab_grouping/model-uxfe87sy:v0")

In [ ]:
len(trainer.tokenizer)

In [ ]:
        tokenized_eval_dataset = tokenized_eval_dataset.map(add_decoder_ids, batched=True)


In [ ]:
        def add_decoder_ids(item_input_dict):
            decoder_input_ids = teacher_model._shift_right(torch.tensor(item_input_dict["labels"]))
            item_input_dict["decoder_input_ids"] = decoder_input_ids


In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    TrainingArguments, Trainer
)


In [ ]:
        teacher_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")


In [ ]:
            item_input_dict["decoder_input_ids"] = decoder_input_ids


In [ ]:
import torch

In [ ]:
        def add_decoder_ids(item_input_dict):
            decoder_input_ids = teacher_model._shift_right(torch.tensor(item_input_dict["labels"]))
            item_input_dict["decoder_input_ids"] = decoder_input_ids
            return item_input_dict
            


In [ ]:
        tokenized_eval_dataset = tokenized_eval_dataset.map(add_decoder_ids, batched=True)


In [ ]:
tokenized_eval_dataset[1]